## 주정차위반이력(df)에 다른 데이터를 병합하는 파일
좌표와 격자와 동이 추가된 주정차위반이력(df)에 <br>
df2 : 자동차등록현황<br>
df3 : 주차장현황<br>
df4 : 건축물대장<br>
df6 : 주택노후도<br>
df7 : 버스정류장<br>
df9 : 동별인구정보<br>
df10 : 공시지가<br>
를 병합

In [1]:
from geoband.API import *
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import folium
import numpy as np
import googlemaps
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from shapely.geometry import Point, Polygon
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('max_rows', 300)
plt.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus=False)

GetCompasData('SBJ_2107_003', '1', '1.부산시남구_주정차위반이력.csv')
GetCompasData('SBJ_2107_003', '2', '2.부산시남구_동별_자동차등록현황.csv')
GetCompasData('SBJ_2107_003', '3', '3.부산시남구_주차장현황')
GetCompasData('SBJ_2107_003', '4', '4.부산시남구_건축물대장(표제부).csv')
GetCompasData('SBJ_2107_003', '5', '5.부산시남구_도로명주소(건물).geojson')
GetCompasData('SBJ_2107_003', '6', '6.부산시남구_주택노후도.geojson')
GetCompasData('SBJ_2107_003', '7', '7.부산시남구_버스정류소정보.csv')
GetCompasData('SBJ_2107_003', '8', '8.부산시남구_지하철역정보.csv')
GetCompasData('SBJ_2107_003', '9', '9.부산시남구_동별_인구정보.csv')
GetCompasData('SBJ_2107_003', '10', '10.부산시남구_공시지가.geojson')
GetCompasData('SBJ_2107_003', '11', '11.부산시남구_도로명주소(도로).geojson')
GetCompasData('SBJ_2107_003', '12', '12.부산시남구_토지이용현황.geojson')
GetCompasData('SBJ_2107_003', '13', '13.부산시남구_토지이용현황(코드).csv')
GetCompasData('SBJ_2107_003', '14', '14.부산시남구_행정경계(읍면동).geojson')
GetCompasData('SBJ_2107_003', '15', '15.부산시남구_격자(300X300).geojson')
GetCompasData('SBJ_2107_003', '16', '16.코드정의서.xlsx')

df1 = pd.read_csv('./1.부산시남구_주정차위반이력.csv')
df2 = pd.read_csv('./2.부산시남구_동별_자동차등록현황.csv')
df3 = pd.read_csv('./3.부산시남구_주차장현황')
df4 = pd.read_csv('./4.부산시남구_건축물대장(표제부).csv')
df5 = json.load(open('5.부산시남구_도로명주소(건물).geojson'))
df6 = json.load(open('./6.부산시남구_주택노후도.geojson'))
df7 = pd.read_csv('./7.부산시남구_버스정류소정보.csv')
df8 = pd.read_csv('./8.부산시남구_지하철역정보.csv')
df9 = pd.read_csv('./9.부산시남구_동별_인구정보.csv')
df10 = json.load(open('./10.부산시남구_공시지가.geojson'))
df11 = json.load(open('./11.부산시남구_도로명주소(도로).geojson'))
df12 = json.load(open('./12.부산시남구_토지이용현황.geojson'))
df13 = pd.read_csv('./13.부산시남구_토지이용현황(코드).csv')
df14 = json.load(open('./14.부산시남구_행정경계(읍면동).geojson'))
df15 = json.load(open('./15.부산시남구_격자(300X300).geojson'))
df16 = pd.read_excel('./16.코드정의서.xlsx')
df18 = gpd.read_file('./14.부산시남구_행정경계(읍면동).geojson')
df19 = gpd.read_file('./15.부산시남구_격자(300X300).geojson')
df = pd.read_csv('./df.csv')

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## 격자의 근사치를 계산하는 코드
격자별로 예측하는 모델을 만들기 위해 격자별로 데이터를 병합해야 함 <br>
shapely를 사용하여 부산시남구_격자 파일의 격자와 정확하게 매핑하는 것은 <br>
시간이 오래 걸리기 때문에 남구의 좌표를 평면으로 근사하여, 30X30으로 격자를 나누고 <br>
이 격자를 기준으로 데이터를 병합

In [2]:
def add_xy(DF):

    long_left = 129.06300949513243
    long_right = 129.1303831414414
    long_interval = (long_right - long_left)/30.0
    lat_top = 35.1613231319683
    lat_bottom = 35.08417426203501
    lat_interval = (lat_top - lat_bottom)/30.0

    x = []
    y = []
    for i in range(len(DF)):
      if (math.ceil(((float(DF['long'].iloc[i]) - long_left)/long_interval)) > 0) and (math.ceil(((float(DF['long'].iloc[i]) - long_left)/long_interval)) <= 30) and (math.ceil((-(float(DF['lat'].iloc[i]) - lat_top)/lat_interval)) > 0) and (math.ceil((-(float(DF['lat'].iloc[i]) - lat_top)/lat_interval)) <= 30) :
        y.append(math.ceil(((float(DF['long'].iloc[i]) - long_left)/long_interval)))
        x.append(math.ceil((-(float(DF['lat'].iloc[i]) - lat_top)/lat_interval)))
      else : 
        y.append(0)
        x.append(0)

    DF['x'] = x
    DF['y'] = y

    DF = DF[DF.x != 0]
    DF = DF[DF.y != 0]

def add_district(DF):
    
    long_left = 129.0586145306224
    long_right = 129.14016727286653
    long_interval = (long_right - long_left)/30.0
    lat_top = 35.16393214819194
    lat_bottom = 35.0885015066011
    lat_interval = (lat_top - lat_bottom)/30.0
    
    x=[]
    y=[]
    for i in range(len(DF)):
      y.append((DF['x'].iloc[i]-0.5) * (long_interval) + long_left)
      x.append(lat_top - ((DF['y'].iloc[i]-0.5) * (long_interval)))

    DF['center_lat'] = y
    DF['center_long'] = x

    district = []

    for i in range(len(DF)):
        district.append((int(DF['y'].iloc[i])-1)*30 + DF['x'].iloc[i])
    
    DF['district'] = district

In [3]:
df

,cd_date,cd_loc,cd_cm_addr,cd_type,lat,long,id,dong
0,2018-09-17,용호동 분포로 하이츠자이 상가 부근,강원도 강릉시 강동면 단경로,스마트폰단속,35.132767,129.110955,608,용호1동
1,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 갈마치로,스마트폰단속,35.132767,129.110955,608,용호1동
2,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 금상로,스마트폰단속,35.132767,129.110955,608,용호1동
3,2018-02-08,용호동 분포로 하이츠자이 상가 부근,경기도 양주시 은현면 화합로,스마트폰단속,35.132767,129.110955,608,용호1동
4,2019-02-01,용호동 분포로 하이츠자이 상가 부근,경상남도 김해시 유하로,스마트폰단속,35.132767,129.110955,608,용호1동
...,...,...,...,...,...,...,...,...
139314,2020-04-20,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동
139315,2020-06-14,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동
139316,2020-09-16,대연동 수영로334번길 56 부근,경상북도 경주시 외동읍 모화남1길,스마트폰단속,35.136148,129.103611,537,대연3동
139317,2020-09-22,대연동 못골번영로71번길 12 부근,경상남도 창원시 마산합포구 노산남4길,스마트폰단속,35.142254,129.084586,289,대연5동


In [4]:
year = []
month = []
for i in range(len(df)):
    year.append(df.iloc[i]['cd_date'][:4])
    month.append(df['cd_date'].iloc[i][5:7])
    
df['year'] = year
df['month'] = month
df['yearmonth'] = df['year'] + df['month']

## 정확한 격자는 id, district는 격자의 근사치
데이터 병합은 id가 아닌 district를 기준으로 병합

In [5]:
add_xy(df)
add_district(df)
df

,cd_date,cd_loc,cd_cm_addr,cd_type,lat,long,id,dong,year,month,yearmonth,x,y,center_lat,center_long,district
0,2018-09-17,용호동 분포로 하이츠자이 상가 부근,강원도 강릉시 강동면 단경로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,09,201809,12,22,129.089876,35.105486,642
1,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 갈마치로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,01,201801,12,22,129.089876,35.105486,642
2,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 금상로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,01,201801,12,22,129.089876,35.105486,642
3,2018-02-08,용호동 분포로 하이츠자이 상가 부근,경기도 양주시 은현면 화합로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,02,201802,12,22,129.089876,35.105486,642
4,2019-02-01,용호동 분포로 하이츠자이 상가 부근,경상남도 김해시 유하로,스마트폰단속,35.132767,129.110955,608,용호1동,2019,02,201902,12,22,129.089876,35.105486,642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139314,2020-04-20,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동,2020,04,202004,20,19,129.111624,35.113641,560
139315,2020-06-14,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동,2020,06,202006,20,19,129.111624,35.113641,560
139316,2020-09-16,대연동 수영로334번길 56 부근,경상북도 경주시 외동읍 모화남1길,스마트폰단속,35.136148,129.103611,537,대연3동,2020,09,202009,10,19,129.084440,35.113641,550
139317,2020-09-22,대연동 못골번영로71번길 12 부근,경상남도 창원시 마산합포구 노산남4길,스마트폰단속,35.142254,129.084586,289,대연5동,2020,09,202009,8,10,129.079003,35.138107,278


## df에 동별자동차등록현황(df2)을 병합
동을 기준으로 병합(용호2동 -> 용호, 대연3동 -> 대연 등) <br>
화물차와 대형차량은 제외

In [6]:
dong = []
for i in range(len(df)):
    dong.append(df.iloc[i]['dong'][:2])    
df['dong_1'] = dong

for i in range(len(df2)):
    if df2.loc[i]['emd_info'][4:]=='화물':
        df2 = df2.drop(i, axis=0)        
df2 = df2.drop(['non_bs_fullsize', 'bs_fullsize'], axis=1)

dong = []
for i in range(len(df2)):
    dong.append(df2.iloc[i]['emd_info'][:2])
df2['dong_1'] = dong

df2_1 = df2.groupby('dong_1').mean()

df2_1.head().T

dong_1,감만,대연,문현,용당,용호
non_bs_city,353.25,1247.00,630.75,192.00,1194.75
non_bs_compact,239.00,796.00,541.00,145.75,698.75
non_bs_midsize,2006.50,8736.00,4114.00,1077.75,7957.50
bs_city,3.50,3.25,4.00,0.50,0.25
bs_compact,12.50,51.75,35.50,5.00,37.50
bs_midsize,123.25,376.50,121.00,81.00,179.00


In [7]:
df = pd.merge(df, df2_1, how='left', on='dong_1')
df.head().T

,0,1,2,3,4
cd_date,2018-09-17,2018-01-11,2018-01-11,2018-02-08,2019-02-01
cd_loc,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근
cd_cm_addr,강원도 강릉시 강동면 단경로,경기도 성남시 중원구 갈마치로,경기도 성남시 중원구 금상로,경기도 양주시 은현면 화합로,경상남도 김해시 유하로
cd_type,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속
lat,35.1328,35.1328,35.1328,35.1328,35.1328
long,129.111,129.111,129.111,129.111,129.111
id,608,608,608,608,608
dong,용호1동,용호1동,용호1동,용호1동,용호1동
year,2018,2018,2018,2018,2019
month,09,01,01,02,02


## df에 주차장현황(df3)를 병합 
district를 기준으로 병합

In [8]:
df3.rename(columns = {'lon' : 'long'}, inplace = True)
add_xy(df3)
add_district(df3)
df3

,park_type_gbn,location,park_slot,long,lat,x,y,center_lat,center_long,district
0,부설,부산광역시 남구 유엔로157번길 10 (대연동 879-14),2,129.093512,35.130656,12,14,129.089876,35.127233,402
1,부설,부산광역시 남구 수영로 274-16 (대연동 372-2),8,129.096972,35.135033,11,16,129.087158,35.121797,461
2,부설,부산광역시 남구 수영로 274-16 (대연동 372-2),8,129.096972,35.135033,11,16,129.087158,35.121797,461
3,부설,부산광역시 남구 유엔로137번길 36 (대연동 1002-28),1,129.090959,35.131716,12,13,129.089876,35.129952,372
4,부설,부산광역시 남구 천제등로16번길 19 (대연동 1219-207),4,129.086612,35.132386,12,11,129.089876,35.135389,312
...,...,...,...,...,...,...,...,...,...,...
4129,민영,부산광역시 남구 우암로 197 (우암동 173-6) 외 2필지,120,129.074503,35.123588,15,6,129.098032,35.148981,165
4130,민영,부산광역시 남구 우암동 265외 1필지,405,129.074691,35.122829,15,6,129.098032,35.148981,165
4131,민영,부산광역시 남구 우암동 173-6,79,129.074270,35.123374,15,6,129.098032,35.148981,165
4132,민영,부산광역시 남구 우암동 174-108,1,129.074613,35.125236,15,6,129.098032,35.148981,165


In [9]:
df = pd.merge(df, pd.DataFrame(df3.groupby('district').mean()['park_slot']), how='left', on='district')
df = pd.merge(df, pd.DataFrame(df3.groupby('district').size()), how='left', on='district')
df = df.rename(columns={0:'park_num'})
df.head().T

,0,1,2,3,4
cd_date,2018-09-17,2018-01-11,2018-01-11,2018-02-08,2019-02-01
cd_loc,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근
cd_cm_addr,강원도 강릉시 강동면 단경로,경기도 성남시 중원구 갈마치로,경기도 성남시 중원구 금상로,경기도 양주시 은현면 화합로,경상남도 김해시 유하로
cd_type,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속
lat,35.1328,35.1328,35.1328,35.1328,35.1328
long,129.111,129.111,129.111,129.111,129.111
id,608,608,608,608,608
dong,용호1동,용호1동,용호1동,용호1동,용호1동
year,2018,2018,2018,2018,2019
month,09,01,01,02,02


In [10]:
df3 = df3[df3['park_type_gbn'] == '공영']
df = pd.merge(df, pd.DataFrame(df3.groupby('district').mean()['park_slot']), how='left', on='district')

In [11]:
m = folium.Map(location=[35.126703, 129.100194], zoom_start=13, prefer_canvas=True)

for i in range(len(df[['lat', 'long', 'district']].groupby('district').mean())):
    if i % 1000:
        folium.Circle(location = [df[['center_lat', 'center_long', 'district']].groupby('district').mean()['center_long'].iloc[i], df[['center_lat', 'center_long', 'district']].groupby('district').mean()['center_lat'].iloc[i]], radius=10, color='red', fill_color='red', fill_opacity=0.1, line_opcity=0.1, tooltip = df[['lat', 'long', 'district']].groupby('district').mean().index[i]).add_to(m)
m

## df에 건축물대장(df4) 병합
in_mec_num    옥내기계식 대수 <br>
in_mec_area    옥내기계식 면적<br>
out_mec_num    옥외 기계식 대수<br>
out_mec_area    옥외 기계식 면적<br>
in_sfp_num    옥내 자주식 대수<br>
in_sfp_area    옥내 자주식 면적<br>
out_sfp_num    옥외 자주식 대수<br>
out_sfp_area    옥외 자주식 면적<br>

In [12]:
df4.rename(columns = {'lon' : 'long'}, inplace = True)
add_xy(df4)
add_district(df4)
df4.head().T

,0,1,2,3,4
loc,부산광역시 남구 대연동 1-4번지,부산광역시 남구 대연동 1-4번지,부산광역시 남구 대연동 10-12번지,부산광역시 남구 대연동 10-2번지,부산광역시 남구 대연동 1002-10번지
sgg_cd,26290,26290,26290,26290,26290
emd_cd,10600,10600,10600,10600,10600
land_gbn,0,0,0,0,0
bun,"""0001""","""0001""","""0010""","""0010""","""1002"""
ji,"""0004""","""0004""","""0012""","""0002""","""0010"""
reg_cd,일반,일반,일반,집합,집합
reg_fld_cd,일반건축물,일반건축물,일반건축물,표제부,표제부
address,,,부산광역시 남구 황령대로 484,부산광역시 남구 황령대로 486,부산광역시 남구 유엔로137번길 46
emd_nm,NaN,NaN,NaN,NaN,대동레미안VI


In [13]:
temp = df4.groupby('district').mean()[['in_mec_num','in_mec_area', 'out_mec_num', 'out_mec_area', 'in_sfp_num', 'in_sfp_area', 
                                       'out_sfp_num', 'out_sfp_area']]
df = pd.merge(df, temp, how='left', on='district')
df.head().T

,0,1,2,3,4
cd_date,2018-09-17,2018-01-11,2018-01-11,2018-02-08,2019-02-01
cd_loc,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근
cd_cm_addr,강원도 강릉시 강동면 단경로,경기도 성남시 중원구 갈마치로,경기도 성남시 중원구 금상로,경기도 양주시 은현면 화합로,경상남도 김해시 유하로
cd_type,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속
lat,35.1328,35.1328,35.1328,35.1328,35.1328
long,129.111,129.111,129.111,129.111,129.111
id,608,608,608,608,608
dong,용호1동,용호1동,용호1동,용호1동,용호1동
year,2018,2018,2018,2018,2019
month,09,01,01,02,02


## df에 주택노후도(df6) 병합

In [14]:
age = []
for i in range(len(df6['features'])):
    age.append(df6['features'][i]['properties']['bd_age'])
    
long6_1 = []
lat6_1 = []
for i in range(len(df6['features'])):
    long6_1.append(df6['features'][i]['geometry']['coordinates'][0][0][0][0])
    lat6_1.append(df6['features'][i]['geometry']['coordinates'][0][0][0][1])

df6_1 = pd.DataFrame({"lat" : lat6_1, "long" : long6_1, "age" : age})

add_xy(df6_1)
add_district(df6_1)
temp = pd.DataFrame(df6_1.groupby('district').mean()['age'])
temp

,age
district,
7,35.922222
8,38.397849
36,31.545455
37,36.091667
38,38.598639
...,...
773,14.000000
774,14.000000
803,14.000000


In [15]:
df = pd.merge(df, temp, how='left', on='district')
df.head().T

,0,1,2,3,4
cd_date,2018-09-17,2018-01-11,2018-01-11,2018-02-08,2019-02-01
cd_loc,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근
cd_cm_addr,강원도 강릉시 강동면 단경로,경기도 성남시 중원구 갈마치로,경기도 성남시 중원구 금상로,경기도 양주시 은현면 화합로,경상남도 김해시 유하로
cd_type,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속
lat,35.1328,35.1328,35.1328,35.1328,35.1328
long,129.111,129.111,129.111,129.111,129.111
id,608,608,608,608,608
dong,용호1동,용호1동,용호1동,용호1동,용호1동
year,2018,2018,2018,2018,2019
month,09,01,01,02,02


## df에 버스정류장(df7) 병합

In [16]:
df7.rename(columns = {'lon' : 'long'}, inplace = True)
add_xy(df7)
add_district(df7)

df = pd.merge(df, pd.DataFrame(df7.groupby('district').size()), how='left', on='district')
df = df.rename(columns={0:'bus_station'})
df.head().T

,0,1,2,3,4
cd_date,2018-09-17,2018-01-11,2018-01-11,2018-02-08,2019-02-01
cd_loc,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근
cd_cm_addr,강원도 강릉시 강동면 단경로,경기도 성남시 중원구 갈마치로,경기도 성남시 중원구 금상로,경기도 양주시 은현면 화합로,경상남도 김해시 유하로
cd_type,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속
lat,35.1328,35.1328,35.1328,35.1328,35.1328
long,129.111,129.111,129.111,129.111,129.111
id,608,608,608,608,608
dong,용호1동,용호1동,용호1동,용호1동,용호1동
year,2018,2018,2018,2018,2019
month,09,01,01,02,02


## df에 동별인구정보(df9) 병합

In [17]:
df9 = df9[df9['age'] == '15~64세']

for i in df9.columns:
    if i[:4] == '2016':
        df9 = df9.drop([i], axis=1)
    if i[:4] == '2017':
        df9 = df9.drop([i], axis=1)

df9_1 = df9[df9['stndrd'] == '계']
df9_2 = df9[df9['stndrd'] == '남']
df9_3 = df9[df9['stndrd'] == '여 ']

temp_1 = []
temp_2 = []
temp_3 = []

for i in range(len(df)):
    if i % 1000 == 0:
        print(str(i)+'/140000')
    temp_1.append(int(df9_1[df9_1['dong'] == df['dong'].iloc[i]][df['yearmonth'].iloc[i]].values))
    temp_2.append(int(df9_2[df9_2['dong'] == df['dong'].iloc[i]][df['yearmonth'].iloc[i]].values))
    temp_3.append(int(df9_3[df9_3['dong'] == df['dong'].iloc[i]][df['yearmonth'].iloc[i]].values))
    
df['계'] = temp_1
df['남'] = temp_2
df['여'] = temp_3
df

0/140000
1000/140000
2000/140000
3000/140000
4000/140000
5000/140000
6000/140000
7000/140000
8000/140000
9000/140000
10000/140000
11000/140000
12000/140000
13000/140000
14000/140000
15000/140000
16000/140000
17000/140000
18000/140000
19000/140000
20000/140000
21000/140000
22000/140000
23000/140000
24000/140000
25000/140000
26000/140000
27000/140000
28000/140000
29000/140000
30000/140000
31000/140000
32000/140000
33000/140000
34000/140000
35000/140000
36000/140000
37000/140000
38000/140000
39000/140000
40000/140000
41000/140000
42000/140000
43000/140000
44000/140000
45000/140000
46000/140000
47000/140000
48000/140000
49000/140000
50000/140000
51000/140000
52000/140000
53000/140000
54000/140000
55000/140000
56000/140000
57000/140000
58000/140000
59000/140000
60000/140000
61000/140000
62000/140000
63000/140000
64000/140000
65000/140000
66000/140000
67000/140000
68000/140000
69000/140000
70000/140000
71000/140000
72000/140000
73000/140000
74000/140000
75000/140000
76000/140000
77000/140000

,cd_date,cd_loc,cd_cm_addr,cd_type,lat,long,id,dong,year,month,yearmonth,x,y,center_lat,center_long,district,dong_1,non_bs_city,non_bs_compact,non_bs_midsize,bs_city,bs_compact,bs_midsize,park_slot_x,park_num,park_slot_y,in_mec_num,in_mec_area,out_mec_num,out_mec_area,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area,age,bus_station,계,남,여
0,2018-09-17,용호동 분포로 하이츠자이 상가 부근,강원도 강릉시 강동면 단경로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,09,201809,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,32514,15881,16633
1,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 갈마치로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,01,201801,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30744,14980,15764
2,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 금상로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,01,201801,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30744,14980,15764
3,2018-02-08,용호동 분포로 하이츠자이 상가 부근,경기도 양주시 은현면 화합로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,02,201802,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30690,14971,15719
4,2019-02-01,용호동 분포로 하이츠자이 상가 부근,경상남도 김해시 유하로,스마트폰단속,35.132767,129.110955,608,용호1동,2019,02,201902,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,32862,16018,16844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139314,2020-04-20,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동,2020,04,202004,20,19,129.111624,35.113641,560,용당,192.00,145.75,1077.75,0.50,5.00,81.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,2.000000,NaN,NaN,5669,2996,2673
139315,2020-06-14,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동,2020,06,202006,20,19,129.111624,35.113641,560,용당,192.00,145.75,1077.75,0.50,5.00,81.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,2.000000,NaN,NaN,5635,2976,2659
139316,2020-09-16,대연동 수영로334번길 56 부근,경상북도 경주시 외동읍 모화남1길,스마트폰단속,35.136148,129.103611,537,대연3동,2020,09,202009,10,19,129.084440,35.113641,550,대연,1247.00,796.00,8736.00,3.25,51.75,376.5,6.494118,85.0,37.0,0.109589,1.286507,0.020548,0.226027,0.938356,10.533171,1.301370,13.592055,26.420290,NaN,24547,11981,12566
139317,2020-09-22,대연동 못골번영로71번길 12 부근,경상남도 창원시 마산합포구 노산남4길,스마트폰단속,35.142254,129.084586,289,대연5동,2020,09,202009,8,10,129.079003,35.138107,278,대연,1247.00,796.00,8736.00,3.25,51.75,376.5,86.000000,1.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,47.777778,555.000000,24.800000,NaN,11925,5867,6058


In [18]:
# df9 = df9[df9['age'] == '15~64세']

# for i in df9.columns:
#     if i[:4] == '2016':
#         df9 = df9.drop([i], axis=1)
#     if i[:4] == '2017':
#         df9 = df9.drop([i], axis=1)

# df9_1 = df9[['201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812']].mean(axis=1)
# df9_2 = df9[['201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912']].mean(axis=1)
# df9_3 = df9[['202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012']].mean(axis=1)
# df9_4 = df9[['202101', '202102', '202103', '202104']].mean(axis=1)

# df9['people_2018'] = df9_1
# df9['people_2019'] = df9_2
# df9['people_2020'] = df9_3
# df9['people_2021'] = df9_4

# df9 = df9[['age', 'dong', 'stndrd', 'people_2018', 'people_2019', 'people_2020', 'people_2021']]

# df9_1 = df9[df9['stndrd'] == '계']
# df9_2 = df9[df9['stndrd'] == '남']
# df9_3 = df9[df9['stndrd'] == '여 ']

# df9_1.columns = ['age', 'dong', 'stndrd', 'people_2018_계', 'people_2019_계', 'people_2020_계', 'people_2021_계']
# df9_2.columns = ['age', 'dong', 'stndrd', 'people_2018_남', 'people_2019_남', 'people_2020_남', 'people_2021_남']
# df9_3.columns = ['age', 'dong', 'stndrd', 'people_2018_여', 'people_2019_여', 'people_2020_여', 'people_2021_여']

# df9_1 = df9_1[['dong', 'people_2018_계', 'people_2019_계', 'people_2020_계', 'people_2021_계']]
# df9_2 = df9_2[['dong', 'people_2018_남', 'people_2019_남', 'people_2020_남', 'people_2021_남']]
# df9_3 = df9_3[['dong', 'people_2018_여', 'people_2019_여', 'people_2020_여', 'people_2021_여']]

# df = pd.merge(df, df9_1, how='left', on='dong')
# df = pd.merge(df, df9_2, how='left', on='dong')
# df = pd.merge(df, df9_3, how='left', on='dong')

# df.head().T

## df에 공시지가(df10) 병합

In [19]:
price = []
lat = []
long = []
for i in range(len(df10['features'])):
    price.append(df10['features'][i]['properties']['lnd_prc'])
    lat.append(df10['features'][i]['geometry']['coordinates'][0][0][0][1])
    long.append(df10['features'][i]['geometry']['coordinates'][0][0][0][0])
    
df10_1 = pd.DataFrame({'price' : price, 'lat' : lat, 'long' : long})
df10_1

,price,lat,long
0,1017000,35.109966,129.113094
1,960000,35.110004,129.113260
2,1160000,35.110036,129.113069
3,726200,35.127279,129.072318
4,0,35.127328,129.072006
...,...,...,...
45772,542600,35.126868,129.069021
45773,571200,35.126962,129.068555
45774,534600,35.125125,129.073358
45775,200900,35.133786,129.066938


In [20]:
add_xy(df10_1)
add_district(df10_1)

df10_1 = df10_1[['price', 'district']]
df = pd.merge(df, df10_1.groupby('district').mean(), how='left', on='district')
df

,cd_date,cd_loc,cd_cm_addr,cd_type,lat,long,id,dong,year,month,yearmonth,x,y,center_lat,center_long,district,dong_1,non_bs_city,non_bs_compact,non_bs_midsize,bs_city,bs_compact,bs_midsize,park_slot_x,park_num,park_slot_y,in_mec_num,in_mec_area,out_mec_num,out_mec_area,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area,age,bus_station,계,남,여,price
0,2018-09-17,용호동 분포로 하이츠자이 상가 부근,강원도 강릉시 강동면 단경로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,09,201809,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,32514,15881,16633,1.179867e+06
1,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 갈마치로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,01,201801,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30744,14980,15764,1.179867e+06
2,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 금상로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,01,201801,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30744,14980,15764,1.179867e+06
3,2018-02-08,용호동 분포로 하이츠자이 상가 부근,경기도 양주시 은현면 화합로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,02,201802,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30690,14971,15719,1.179867e+06
4,2019-02-01,용호동 분포로 하이츠자이 상가 부근,경상남도 김해시 유하로,스마트폰단속,35.132767,129.110955,608,용호1동,2019,02,201902,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,32862,16018,16844,1.179867e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139314,2020-04-20,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동,2020,04,202004,20,19,129.111624,35.113641,560,용당,192.00,145.75,1077.75,0.50,5.00,81.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,2.000000,NaN,NaN,5669,2996,2673,1.125143e+05
139315,2020-06-14,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동,2020,06,202006,20,19,129.111624,35.113641,560,용당,192.00,145.75,1077.75,0.50,5.00,81.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,2.000000,NaN,NaN,5635,2976,2659,1.125143e+05
139316,2020-09-16,대연동 수영로334번길 56 부근,경상북도 경주시 외동읍 모화남1길,스마트폰단속,35.136148,129.103611,537,대연3동,2020,09,202009,10,19,129.084440,35.113641,550,대연,1247.00,796.00,8736.00,3.25,51.75,376.5,6.494118,85.0,37.0,0.109589,1.286507,0.020548,0.226027,0.938356,10.533171,1.301370,13.592055,26.420290,NaN,24547,11981,12566,2.250281e+06
139317,2020-09-22,대연동 못골번영로71번길 12 부근,경상남도 창원시 마산합포구 노산남4길,스마트폰단속,35.142254,129.084586,289,대연5동,2020,09,202009,8,10,129.079003,35.138107,278,대연,1247.00,796.00,8736.00,3.25,51.75,376.5,86.000000,1.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,47.777778,555.000000,24.800000,NaN,11925,5867,6058,4.342857e+04


In [21]:
df.head().T

,0,1,2,3,4
cd_date,2018-09-17,2018-01-11,2018-01-11,2018-02-08,2019-02-01
cd_loc,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근,용호동 분포로 하이츠자이 상가 부근
cd_cm_addr,강원도 강릉시 강동면 단경로,경기도 성남시 중원구 갈마치로,경기도 성남시 중원구 금상로,경기도 양주시 은현면 화합로,경상남도 김해시 유하로
cd_type,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속,스마트폰단속
lat,35.1328,35.1328,35.1328,35.1328,35.1328
long,129.111,129.111,129.111,129.111,129.111
id,608,608,608,608,608
dong,용호1동,용호1동,용호1동,용호1동,용호1동
year,2018,2018,2018,2018,2019
month,09,01,01,02,02


In [22]:
df.to_csv('total.csv', index=False)

In [23]:
df = pd.read_csv('./total.csv')
df

,cd_date,cd_loc,cd_cm_addr,cd_type,lat,long,id,dong,year,month,yearmonth,x,y,center_lat,center_long,district,dong_1,non_bs_city,non_bs_compact,non_bs_midsize,bs_city,bs_compact,bs_midsize,park_slot_x,park_num,park_slot_y,in_mec_num,in_mec_area,out_mec_num,out_mec_area,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area,age,bus_station,계,남,여,price
0,2018-09-17,용호동 분포로 하이츠자이 상가 부근,강원도 강릉시 강동면 단경로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,9,201809,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,32514,15881,16633,1.179867e+06
1,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 갈마치로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,1,201801,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30744,14980,15764,1.179867e+06
2,2018-01-11,용호동 분포로 하이츠자이 상가 부근,경기도 성남시 중원구 금상로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,1,201801,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30744,14980,15764,1.179867e+06
3,2018-02-08,용호동 분포로 하이츠자이 상가 부근,경기도 양주시 은현면 화합로,스마트폰단속,35.132767,129.110955,608,용호1동,2018,2,201802,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,30690,14971,15719,1.179867e+06
4,2019-02-01,용호동 분포로 하이츠자이 상가 부근,경상남도 김해시 유하로,스마트폰단속,35.132767,129.110955,608,용호1동,2019,2,201902,12,22,129.089876,35.105486,642,용호,1194.75,698.75,7957.50,0.25,37.50,179.0,819.333333,3.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,7.666667,16.363636,NaN,32862,16018,16844,1.179867e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139314,2020-04-20,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동,2020,4,202004,20,19,129.111624,35.113641,560,용당,192.00,145.75,1077.75,0.50,5.00,81.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,2.000000,NaN,NaN,5669,2996,2673,1.125143e+05
139315,2020-06-14,용호동 신선대산복로 203 부근,울산광역시 울주군 범서읍 굴화1길,스마트폰단속,35.111950,129.103475,548,용당동,2020,6,202006,20,19,129.111624,35.113641,560,용당,192.00,145.75,1077.75,0.50,5.00,81.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,2.000000,NaN,NaN,5635,2976,2659,1.125143e+05
139316,2020-09-16,대연동 수영로334번길 56 부근,경상북도 경주시 외동읍 모화남1길,스마트폰단속,35.136148,129.103611,537,대연3동,2020,9,202009,10,19,129.084440,35.113641,550,대연,1247.00,796.00,8736.00,3.25,51.75,376.5,6.494118,85.0,37.0,0.109589,1.286507,0.020548,0.226027,0.938356,10.533171,1.301370,13.592055,26.420290,NaN,24547,11981,12566,2.250281e+06
139317,2020-09-22,대연동 못골번영로71번길 12 부근,경상남도 창원시 마산합포구 노산남4길,스마트폰단속,35.142254,129.084586,289,대연5동,2020,9,202009,8,10,129.079003,35.138107,278,대연,1247.00,796.00,8736.00,3.25,51.75,376.5,86.000000,1.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,47.777778,555.000000,24.800000,NaN,11925,5867,6058,4.342857e+04
